In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor , GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
df_train = pd.read_csv('./all/train.csv')
df_test = pd.read_csv('./all/test.csv')
df = pd.concat((df_train.loc[:, 'Id' : 'SaleCondition'], df_test.loc[:, 'Id' : 'SaleCondition']))
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
train_min = min(df_train.Id)
train_max = max(df_train.Id)
test_min = min(df_test.Id)
test_max = max(df_test.Id)

In [4]:
df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 80 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null int64
MSZoning         2915 non-null object
LotFrontage      2433 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            198 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2917 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2918 non-

In [6]:
def apply_preprocessing_to_data(df, drop_threshold, drop_first, dummy):
    
    
    cat_variables = []
    num_variables = []
    columns_to_be_dropped=[]
    
    empty_columns = df.columns[df.isna().any()].tolist()
    for i in empty_columns:
        add = df[i].isnull().sum()
        if((add / len(df) *100) > drop_threshold):
            columns_to_be_dropped.append(i)

    first_column = df.columns[0]
    
    if(drop_first):
        df =df.drop(axis =1 , columns= df.columns[0])

    df =  df.drop(axis = 1 , columns = columns_to_be_dropped)
    
    
    
    
    for column in df.columns:
        if(df[column].dtype == 'O'):
            cat_variables.append(column)
        else:
            num_variables.append(column)
    
    for i in cat_variables:
        if i in empty_columns:
            top_variable = df[i].describe().top
            df[i] = df[i].fillna(top_variable)

    for i in num_variables:
        if i in empty_columns:
            mean_ = df[i].mean()
            df[i] = df[i].fillna(mean_)

    if(dummy):
        df = pd.get_dummies(df, columns = cat_variables)
        
    return df


In [7]:
df = apply_preprocessing_to_data(df, 50 , False, True)
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,0,1,0,0,0,0,1,0


In [8]:
def Get_X_y(df, target_variable, train_max, train_min, test_min, test_max):
    X_train  = df[((df.Id >= train_min) & (df.Id <= train_max))]
    y_train =  df_train[target_variable]
    
    X_test = df[((df.Id >= test_min) & (df.Id <= test_max))]
    
    X_train  =X_train.drop(columns = ['Id'])
    X_test  =X_test.drop(columns = ['Id'])
    
    X = np.array(X_train)
    y = np.array(y_train)
    
    return X,y, X_test

In [9]:
X ,y, X_test = Get_X_y(df, 'SalePrice', train_max, train_min, test_min, test_max)
X.shape, y.shape, X_test.shape

((1460, 275), (1460,), (1459, 275))

In [10]:

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1, shuffle=True)
X_train.shape

(1314, 275)

In [11]:
def apply_advanced_preprocessing( pca, pca_variance , X_train, X_valid, X_test, scale=True):
    if(scale):
        min_max = MinMaxScaler()
        min_max.fit(X_train)
        X_train = min_max.transform(X_train)
        X_valid = min_max.transform(X_valid)
        X_test = min_max.transform(X_test)
    
    if(pca):
        pca = PCA(pca_variance).fit(X_train)
        X_train =  pca.transform(X_train)
        X_valid = pca.transform(X_valid)
        X_test = pca.transform(X_test)
        
    return X_train, X_valid, X_test

In [12]:
X_train, X_valid, X_test = apply_advanced_preprocessing(True, 0.95, X_train, X_valid, X_test, True)


In [13]:
X_train.shape, X_test.shape

((1314, 99), (1459, 99))

In [14]:
def err(y_test, y_pred):
    return mean_squared_log_error(y_test, y_pred)

In [15]:
def GridSearch(classifier):
    Cs = [0.001, 0.01, 0.1, 1, 10,1000]
    gammas = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'gamma' : gammas}

    grid_search = GridSearchCV(classifier, param_grid)
    grid_search.fit(X_train, y_train)
    
    return grid_search.best_params_


In [16]:
GridSearch(SVR())

{'C': 1000, 'gamma': 0.01}

In [17]:
def GridSearch_New(classifier):
    Cs = [0.001, 0.01, 0.1, 1, 10,1000]
    gammas = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'gamma' : gammas}

    grid_search = GridSearchCV(classifier, param_grid)
    grid_search.fit(X_train, y_train)
    
    return grid_search.best_params_

In [18]:
ada = AdaBoostRegressor().fit(X_train, y_train)
gra = GradientBoostingRegressor().fit(X_train, y_train)
rf = RandomForestRegressor().fit(X_train, y_train)
xgb = XGBRegressor().fit(X_train,y_train)

In [19]:
svr = SVR(C=1000, gamma=0.01)

In [20]:
svr.fit(X_train, y_train)

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.01,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [21]:
def predict_classifier(classifier_list,X,y_valid=None):
    pred = []
    for classifier in classifier_list:
        y_pred = classifier.predict(X)
        if(y_valid is not None):
            print(err(y_valid, y_pred))
        pred.append(y_pred)
    return np.mean(pred, axis = 0)

In [73]:
final_pred = predict_classifier ([ xgb], X_valid,y_valid)

0.039961372016954926


In [74]:
print(err(y_valid, final_pred))

0.039961372016954926


In [75]:
y_final_pred = predict_classifier([ xgb] , X_test)

In [76]:
ss = pd.read_csv('./all/sample_submission.csv')
ss.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [77]:
ss.loc[:, 'SalePrice']  = y_final_pred

In [78]:
y_final_pred

array([122626.03, 152875.52, 178764.89, ..., 137882.22, 129158.74,
       209810.14], dtype=float32)

In [79]:
ss.to_csv('version4.csv',index=False)

In [66]:
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras import backend as K

In [71]:
model = Sequential()
model.add(BatchNormalization(input_shape = (99,)))
model.add(Dense(50,  activation='elu'))

model.add(BatchNormalization())
model.add(Dense(25, activation='elu'))
model.add(Dense(1))

model.compile(optimizer = Adam(lr = 0.001) , loss = 'mse')

In [72]:
model.fit(X_train, y_train, epochs = 1000 , validation_data=(X_valid,y_valid), shuffle=True)

Train on 1314 samples, validate on 146 samples
Epoch 1/1000
1314/1314 [==============================] - 0s 357us/step - loss: 38878743840.3409 - val_loss: 40483990261.4795
Epoch 2/1000
1314/1314 [==============================] - 0s 82us/step - loss: 38878186488.2070 - val_loss: 40483268916.6027
Epoch 3/1000
1314/1314 [==============================] - 0s 82us/step - loss: 38877180948.2618 - val_loss: 40481734599.8904
Epoch 4/1000
1314/1314 [==============================] - 0s 87us/step - loss: 38874961549.0533 - val_loss: 40478839232.8767
Epoch 5/1000
1314/1314 [==============================] - 0s 86us/step - loss: 38870210669.1020 - val_loss: 40474082710.7945
Epoch 6/1000
1314/1314 [==============================] - 0s 82us/step - loss: 38861020501.3333 - val_loss: 40464325309.3699
Epoch 7/1000
1314/1314 [==============================] - 0s 88us/step - loss: 38844773312.0974 - val_loss: 40447903098.7397
Epoch 8/1000
1314/1314 [==============================] - 0s 80us/step - loss

1314/1314 [==============================] - 0s 83us/step - loss: 19829618853.2116 - val_loss: 20502870605.1507
Epoch 66/1000
1314/1314 [==============================] - 0s 81us/step - loss: 19269914390.2100 - val_loss: 19538572302.0274
Epoch 67/1000
1314/1314 [==============================] - 0s 80us/step - loss: 18767324691.4825 - val_loss: 19008952474.3014
Epoch 68/1000
1314/1314 [==============================] - 0s 81us/step - loss: 18254054150.6241 - val_loss: 19642216644.3836
Epoch 69/1000
1314/1314 [==============================] - 0s 83us/step - loss: 17770693256.3775 - val_loss: 19036653259.3973
Epoch 70/1000
1314/1314 [==============================] - 0s 82us/step - loss: 17170324045.9300 - val_loss: 18219215886.0274
Epoch 71/1000
1314/1314 [==============================] - 0s 84us/step - loss: 16730012927.6103 - val_loss: 17069554421.4795
Epoch 72/1000
1314/1314 [==============================] - 0s 81us/step - loss: 16207381550.7580 - val_loss: 15943782512.2192
Epoch 

1314/1314 [==============================] - 0s 86us/step - loss: 3186346317.5403 - val_loss: 3188431803.6164
Epoch 131/1000
1314/1314 [==============================] - 0s 84us/step - loss: 3135228314.6910 - val_loss: 3151884743.8904
Epoch 132/1000
1314/1314 [==============================] - 0s 81us/step - loss: 3272693948.5906 - val_loss: 3152348088.1096
Epoch 133/1000
1314/1314 [==============================] - 0s 86us/step - loss: 3297561672.0852 - val_loss: 3071834958.9041
Epoch 134/1000
1314/1314 [==============================] - 0s 83us/step - loss: 3248835011.4094 - val_loss: 3281500689.5342
Epoch 135/1000
1314/1314 [==============================] - 0s 79us/step - loss: 2953669362.3623 - val_loss: 3048158050.1918
Epoch 136/1000
1314/1314 [==============================] - 0s 80us/step - loss: 3050620330.2770 - val_loss: 3185713292.2740
Epoch 137/1000
1314/1314 [==============================] - 0s 81us/step - loss: 2949295376.7549 - val_loss: 3084244425.6438
Epoch 138/1000


1314/1314 [==============================] - 0s 80us/step - loss: 1501924288.0974 - val_loss: 2080929102.9041
Epoch 196/1000
1314/1314 [==============================] - 0s 81us/step - loss: 1454338458.3014 - val_loss: 2070861976.5479
Epoch 197/1000
1314/1314 [==============================] - 0s 81us/step - loss: 1457107633.6804 - val_loss: 1931507299.9452
Epoch 198/1000
1314/1314 [==============================] - 0s 82us/step - loss: 1451123158.3075 - val_loss: 1859476988.4932
Epoch 199/1000
1314/1314 [==============================] - 0s 103us/step - loss: 1457522874.2527 - val_loss: 1788997112.9863
Epoch 200/1000
1314/1314 [==============================] - 0s 81us/step - loss: 1287095501.3455 - val_loss: 1754486896.2192
Epoch 201/1000
1314/1314 [==============================] - 0s 79us/step - loss: 1409394145.6073 - val_loss: 1694936497.9726
Epoch 202/1000
1314/1314 [==============================] - 0s 82us/step - loss: 1296709928.1339 - val_loss: 1677599008.4384
Epoch 203/1000

Epoch 261/1000
1314/1314 [==============================] - 0s 94us/step - loss: 1003059668.3592 - val_loss: 1340292583.4521
Epoch 262/1000
1314/1314 [==============================] - 0s 85us/step - loss: 1011902615.1842 - val_loss: 1338457624.5479
Epoch 263/1000
1314/1314 [==============================] - 0s 77us/step - loss: 1037322671.0989 - val_loss: 1333865455.3425
Epoch 264/1000
1314/1314 [==============================] - 0s 80us/step - loss: 989549409.8021 - val_loss: 1339293836.2740
Epoch 265/1000
1314/1314 [==============================] - 0s 82us/step - loss: 892828636.9315 - val_loss: 1339991651.9452
Epoch 266/1000
1314/1314 [==============================] - 0s 82us/step - loss: 943924430.5145 - val_loss: 1335888317.3699
Epoch 267/1000
1314/1314 [==============================] - 0s 88us/step - loss: 949298713.7169 - val_loss: 1336915112.3288
Epoch 268/1000
1314/1314 [==============================] - 0s 94us/step - loss: 955156013.1994 - val_loss: 1351308791.2329
Epoch

Epoch 327/1000
1314/1314 [==============================] - 0s 83us/step - loss: 929913699.9452 - val_loss: 1305035212.2740
Epoch 328/1000
1314/1314 [==============================] - 0s 81us/step - loss: 870287449.0350 - val_loss: 1311514783.5616
Epoch 329/1000
1314/1314 [==============================] - 0s 82us/step - loss: 910714783.1720 - val_loss: 1338109710.0274
Epoch 330/1000
1314/1314 [==============================] - 0s 79us/step - loss: 898709219.5556 - val_loss: 1336800008.7671
Epoch 331/1000
1314/1314 [==============================] - 0s 80us/step - loss: 892993764.3349 - val_loss: 1308720625.0959
Epoch 332/1000
1314/1314 [==============================] - 0s 79us/step - loss: 949042205.2237 - val_loss: 1316383773.8082
Epoch 333/1000
1314/1314 [==============================] - 0s 80us/step - loss: 882011889.9726 - val_loss: 1309898337.3151
Epoch 334/1000
1314/1314 [==============================] - 0s 79us/step - loss: 943453794.5419 - val_loss: 1285687330.1918
Epoch 33

1314/1314 [==============================] - 0s 80us/step - loss: 758509462.7945 - val_loss: 1277145463.2329
Epoch 394/1000
1314/1314 [==============================] - 0s 80us/step - loss: 765361797.6499 - val_loss: 1280984483.0685
Epoch 395/1000
1314/1314 [==============================] - 0s 84us/step - loss: 795439354.1553 - val_loss: 1263091975.8904
Epoch 396/1000
1314/1314 [==============================] - 0s 82us/step - loss: 729118521.4734 - val_loss: 1283778613.9178
Epoch 397/1000
1314/1314 [==============================] - 0s 80us/step - loss: 726774551.5251 - val_loss: 1264834572.2740
Epoch 398/1000
1314/1314 [==============================] - 0s 82us/step - loss: 769983572.2618 - val_loss: 1263402456.9863
Epoch 399/1000
1314/1314 [==============================] - 0s 93us/step - loss: 881239566.4170 - val_loss: 1245675190.7945
Epoch 400/1000
1314/1314 [==============================] - 0s 94us/step - loss: 794051918.3196 - val_loss: 1264422409.2055
Epoch 401/1000
1314/131

KeyboardInterrupt: 